<a href="https://colab.research.google.com/github/mohamedahmedsaadahmed77/Sarcasm-Detection/blob/main/Arabic_Sarcasm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import string
import re
from nltk.corpus import stopwords
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import make_pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import SVC
from sklearn.metrics import confusion_matrix,accuracy_score, classification_report
import nltk
nltk.download('stopwords')

#/content/gdrive/MyDrive/Colab Notebooks/selected3/ArSarcasm_train.csv
!git clone https://github.com/mohamedahmedsaadahmed77/Sarcasm_Detection.git
data = pd.read_csv("/content/Sarcasm_Detection/ArSarcasm_train.csv")
data

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
Cloning into 'Sarcasm_Detection'...
remote: Enumerating objects: 16, done.
remote: Counting objects: 100% (16/16), done.
remote: Compressing objects: 100% (15/15), done.
remote: Total 16 (delta 7), reused 5 (delta 0), pack-reused 0
Unpacking objects: 100% (16/16), done.


,dialect,sarcasm,sentiment,original_sentiment,tweet,source
0,gulf,False,negative,negative,"""نصيحه ما عمرك اتنزل لعبة سوبر ماريو مش زي ما ...",semeval
1,msa,False,neutral,positive,"""#نادين_نسيب_نجيم ❤️❤️❤️مجلة #ماري_كلير 💭#ملكة...",semeval
2,egypt,False,neutral,neutral,"""@Alito_NBA اتوقع انه بيستمر""",semeval
3,levant,True,neutral,negative,"""@KSA24 يعني ""بموافقتنا"" لأن دمشق صايرة موسكو""",semeval
4,msa,False,neutral,negative,"""RT @alaahmad20: قائد في الحرس يعترف بفقدان ال...",semeval
...,...,...,...,...,...,...
8432,msa,False,neutral,neutral,"""RT @iDhmi: #توصيات_تويتسو | تطبيق يوتيوب الأط...",semeval
8433,msa,False,neutral,neutral,سي بي سي:#إبراهيم_محلب يشارك باحتفالية تنزانيا...,astd
8434,msa,True,negative,negative,"""كلب إسرائيل وسيفها الإرهابي بوتن قاتل النساء ...",semeval
8435,msa,True,negative,negative,"""احمد فتفت صار مغضوب عليه بمجرد اعترضو على مي...",semeval


In [2]:
punctuations = '''`÷×؛<>_()*&^%][ـ،/:"؟.,'{}~¦+|!”…“–ـ''' + string.punctuation

# Arabic stop words with nltk
stop_words = stopwords.words()

arabic_diacritics = re.compile("""
                             ّ    | # Shadda
                             َ    | # Fatha
                             ً    | # Tanwin Fath
                             ُ    | # Damma
                             ٌ    | # Tanwin Damm
                             ِ    | # Kasra
                             ٍ    | # Tanwin Kasr
                             ْ    | # Sukun
                             ـ     # Tatwil/Kashida
                         """, re.VERBOSE)

In [3]:
def preprocess(text):
    
    '''
    text is an arabic string input
    
    the preprocessed text is returned
    '''
    text = re.sub(r"http\S+", " ", text)


    #remove punctuations

    translator = str.maketrans('', '', punctuations)
    text = text.translate(translator)
    
    # remove Tashkeel
    text = re.sub(arabic_diacritics, '', text)
    
    #remove longation
    text = re.sub("[إأآا]", "ا", text)
    text = re.sub("ى", "ي", text)
    text = re.sub("ؤ", "ء", text)
    text = re.sub("ئ", "ء", text)
    text = re.sub("ة", "ه", text)
    text = re.sub("گ", "ك", text)

    text = [word for word in text.split() if word not in stop_words]

    return text

In [4]:
feature = [preprocess(x) for x in data.tweet]
target = data.sarcasm
# splitting into train and tests
X_train, X_test, Y_train, Y_test = train_test_split(feature, target, test_size =.2, random_state=100)

In [5]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

import re
import string
import csv


import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer

nltk.download('punkt')
nltk.download('stopwords')

class TextNBClassifier:
    def __init__(self):
        pass
    
    def count_words_freq(self, xs, ys):
      freqs = {}
      for words, y in zip(xs, ys):
          for word in words:
              pair = (word,y)
              if pair in freqs:
                  freqs[pair] += 1
              else:
                  freqs[pair] = 1
      return freqs

    def lookup(self, freqs, word, is_sarcastic):
      if (word, is_sarcastic) in freqs:
        return freqs[(word, is_sarcastic)]
      else:
        return 0
      return freq

    def fit(self, train_x, train_y):
      freqs = self.count_words_freq(train_x, train_y)
      self.loglikelihood = {}
      self.logprior = 0

      vocab = set([pair[0] for pair in freqs.keys()])
      V = len(vocab)

      N_pos = N_neg = V_pos = V_neg = 0
      for pair in freqs.keys():
          if pair[1] > 0:
              V_pos += 1
              N_pos += freqs[pair]
          else:
              V_neg += 1
              N_neg += freqs[pair]
      D = len(train_y)
      
      D_pos = (len(list(filter(lambda x: x > 0, train_y))))
      D_neg = (len(list(filter(lambda x: x <= 0, train_y))))

      logprior = np.log(D_pos) - np.log(D_neg)

      for word in vocab:
          freq_pos = self.lookup(freqs,word,1)
          freq_neg = self.lookup(freqs,word,0)

          p_w_pos = (freq_pos + 1) / (N_pos + V)
          p_w_neg = (freq_neg + 1) / (N_neg + V)

          self.loglikelihood[word] = np.log(p_w_pos/p_w_neg)
      return self

    def predict_per_record(self, word_l):
      p = 0
      p += self.logprior

      for word in word_l:
        if word in self.loglikelihood:
              p += self.loglikelihood[word]
      return p

    def predict(self, test_x, test_y):
      self.accuracy = 0
      y_hats = []
      for word_l in test_x:
          if self.predict_per_record(word_l) > 0:
              y_hat_i = 1
          else:
              y_hat_i = 0
          y_hats.append(y_hat_i)
      error = np.mean(np.absolute(y_hats-test_y))
      self.accuracy = 1-error    
      return self

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [6]:
nbc = TextNBClassifier()
nbc.fit(X_train,Y_train)
nbc.predict(X_train, Y_train)
print("Training Accuracy : %.2f"%(nbc.accuracy * 100))

Training Accuracy : 96.04


In [7]:
nbc.predict(X_test, Y_test)
print("Testing Accuracy : %.2f"%(nbc.accuracy * 100))

Testing Accuracy : 83.06
